In [1]:
import pandas as pd
import json

import numpy as np
import faiss
from langchain.docstore.document import Document
from langchain.document_loaders import JSONLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [32]:
def load_bias_terms(file_path: str) -> list[Document]:
    """
    Load the bias terms JSON file and return a list of Document objects.
    
    Each document will contain the bias type and its description.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    documents = []
    for entry in data:
        bias_type = entry.get("Bias_Type", "Unknown Bias")
        description = entry.get("Description", "")
        content = f"Bias: {bias_type}\nDescription: {description}"
        documents.append(Document(page_content=content, metadata={"type":"definition", "source": file_path}))
    
    return documents

def load_document_text(file_path: str) -> list[Document]:
    
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    documents = []
    for entry in data:
        country = entry.get("country", "Unknown country")
        doc_nr = entry.get("document_nr", "")
        text_chunk = entry.get("text", "")
        content = f"Country: {country}\nDocument number: {doc_nr}\nSentence: {text_chunk}"
        documents.append(Document(page_content=content, metadata={"type":"document text", "source": file_path}))
    return documents
    


# Example usage:
bias_documents = load_bias_terms("RAG_data/bias_terms.json")
text_documents = load_document_text("RAG_data/documents_text.json")

print(f"Loaded {len(bias_documents)} documents.")
print(text_documents[0])

Loaded 65 documents.
page_content='Country: CHL
Document number: 1
Sentence: POLTICA NACIONAL DE INTELIGENCIA ARTIFICIALPOLTICA NACIONAL DE INTELIGENCIA ARTIFICIALCOMIT DE EXPERTOSCOORDINACIN REDACCIN Y EDICIN COMUNICACIONES  Y DISEOCOMIT INTERMINISTERIAL Marcelo ArenasPontificia Universidad Catlica de Chile e IM Fundamentos de los Datos Nstor BecerraUniversidad de Chile Raphael BergoeingUniversidad de Chile y Comisin Nacional de Productividad Alberto CerdaUniversidad de Chile Aisn EtcheverryAgencia Nacional de Investigacin y DesarrolloJos A' metadata={'type': 'document text', 'source': 'RAG_data/documents_text.json'}


In [ ]:
# Initialize the BERT tokenizer (using the 'bert-base-uncased' model)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# A simple function to load a JSON file and convert it to a LangChain Document.
def load_json_to_document(file_path: str) -> Document:
    with open(file_path, 'r') as f:
        data = json.load(f)
    # Assume the JSON contains a field "text". If not, convert the whole JSON to a string.
    text = data.get("text", str(data))
    return Document(page_content=text, metadata={"source": file_path})

# List your JSON file paths
doc_paths = ["doc1.json", "doc2.json", "doc3.json"]

# Load documents from the JSON files
documents = [load_json_to_document(path) for path in doc_paths]

# Optional: Function to tokenize a document with BERT and split into smaller chunks if needed.
def tokenize_document(doc: Document, max_tokens: int = 128):
    tokens = tokenizer.tokenize(doc.page_content)
    # If the document is longer than max_tokens, split it into chunks.
    if len(tokens) > max_tokens:
        chunks = []
        for i in range(0, len(tokens), max_tokens):
            chunk_tokens = tokens[i:i+max_tokens]
            chunk_text = tokenizer.convert_tokens_to_string(chunk_tokens)
            chunks.append(Document(page_content=chunk_text, metadata=doc.metadata))
        return chunks
    else:
        return [doc]

# Process each document (this step is optional if you don't require chunking)
processed_docs = []
for doc in documents:
    processed_docs.extend(tokenize_document(doc))

# Initialize HuggingFace embeddings (you can choose a model based on your needs)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create a FAISS vector store from the processed documents
vectorstore = FAISS.from_documents(processed_docs, embeddings)

# Convert the vectorstore into a retriever (here, returning the top-3 nearest neighbors)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Initialize your LLM (OpenAI's model in this case; ensure your API key is set in your environment)
llm = OpenAI(temperature=0)

# Create the RetrievalQA (RAG) chain tying the retriever and the LLM together
qa_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever)

# Query the RAG system
query = "What are the key points discussed in the documents?"
result = qa_chain.run(query)
print(result)

In [ ]:
bias_terms_loader = JSONLoader("RAG_data/bias_terms.json", jq_schema=".[] | {page_content: .Description, metadata: {Bias_Type: .Bias_Type}}")
document_metrics_loader = JSONLoader("RAG_data/document_metrics.json", jq_schema=".")
documents_text_loader = JSONLoader("RAG_data/documents_text.json", jq_schema=".")

bias_terms = bias_terms_loader.load()
#document_metrics = document_metrics_loader.load()
#documents_text = documents_text_loader.load()

# Combine the documents into one list
documents = bias_terms + document_metrics + documents_text

# Initialize HuggingFaceEmbeddings with a BERT model.
# This will automatically use the BERT tokenizer from Hugging Face.
embeddings = HuggingFaceEmbeddings(model_name="bert-base-uncased")

# Build a FAISS vectorstore from the documents
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Initialize your LLM and set up the RetrievalQA chain
llm = OpenAI(temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever)

# Run a query
query = "What are the most important types biases discussed in the USA policy documents"
result = qa_chain.run(query)
print(result)

ValueError: Expected page_content is string, got <class 'dict'> instead.                     Set `text_content=False` if the desired input for                     `page_content` is not a string